events
1. Arrival of order to company from supplier
2. Demand for product from a customer
3. End of the simulation ($n$ months)
4. Inventory evaluation and possible ordering at the beginning of a month

$$ \text{interdemand times} = S_i \sim \text{Exp} $$

$$ \text{demand-size} = D \sim p\left(D\right)$$

In [7]:
import numpy as np

def generate_interdemand_time(mean):
    return np.random.exponential(scale=mean)

def generate_demand_size():
    return np.random.choice(a=[1,2,3,4], p=[1/6, 1/3, 1/3, 1/6])

def generate_delivery_lag():
    return np.random.uniform(0.5, 1)

#### Order Arrival Event

Make changes necessary when an order arrives from the supplier.
- Inventory level is increased
- Order arrival event is eliminated from consideration

#### Demand Event

Process changes necessary to represent a demand's occurrence.
- Demand size is generated
- Inventory is decremented accordingly (may become negative!)
- Next demand event is scheduled in event list

#### Evaluation Event

At start of every month, evaluate the inventory and place an order accordingly.
- If $I\left(t\right) \geq s$, no order is placed.
- If $I\left(t\right) < s$, place an order for $S-I\left(t\right)$ iterms.
    - Store the amount of order $\left[S - I\left(t\right)\right]$ until the order arrives
    - Schedule its arrival time
- Schedule the next inventory-evaluation event.

In [51]:
import matplotlib.pyplot as plt
import numpy as np

def generate_interdemand_time(mean=0.10):
    return np.random.exponential(scale=mean)

def generate_demand_size():
    return np.random.choice(a=[1,2,3,4], p=[1/6, 1/3, 1/3, 1/6])

def generate_delivery_lag():
    return np.random.uniform(0.5, 1)


INITIAL_INV_LEVEL = 10
NUM_MONTHS = 3


def initialize():
    global SIM_TIME, INV_LEVEL, TIME_LAST_EVENT, TOTAL_ORDERING_COST, AREA_HOLDING, AREA_SHORTAGE, TIME_NEXT_EVENT

    SIM_TIME = 0.0

    global INITIAL_INV_LEVEL
    INV_LEVEL = INITIAL_INV_LEVEL
    TIME_LAST_EVENT = 0.0

    TOTAL_ORDERING_COST = 0.0
    AREA_HOLDING = 0.0
    AREA_SHORTAGE = 0.0

    global NUM_MONTHS
    TIME_NEXT_EVENT = [
        np.inf,
        SIM_TIME + generate_interdemand_time(),
        NUM_MONTHS,
        0.0
    ]


def order_arrival():
    global INV_LEVEL, AMOUNT, TIME_NEXT_EVENT
    INV_LEVEL += AMOUNT
    TIME_NEXT_EVENT[0] = np.inf

def demand():
    global INV_LEVEL, TIME_NEXT_EVENT, SIM_TIME
    INV_LEVEL -= generate_demand_size()
    TIME_NEXT_EVENT[1] = SIM_TIME + generate_interdemand_time()

def evaluate():
    global INV_LEVEL, SMALL_S, TIME_NEXT_EVENT, SIM_TIME
    if INV_LEVEL < SMALL_S:
        # place an order
        global AMOUNT, BIG_S, TOTAL_ORDERING_COST, SETUP_COST, INCREMENTAL_COST
        AMOUNT = BIG_S - INV_LEVEL
        TOTAL_ORDERING_COST += SETUP_COST + INCREMENTAL_COST * AMOUNT

        # schedule order arrival
        TIME_NEXT_EVENT[0] = SIM_TIME + generate_delivery_lag()

    # schedule next evaluation
    TIME_NEXT_EVENT[3] = SIM_TIME + 1.0


def report():
    global TOTAL_ORDERING_COST, NUM_MONTHS, AREA_HOLDING, HOLDING_COST, AREA_SHORTAGE, SHORTAGE_COST
    global SMALL_S, BIG_S
    avg_ordering_cost = TOTAL_ORDERING_COST / NUM_MONTHS
    avg_holding_cost = HOLDING_COST * AREA_HOLDING / NUM_MONTHS
    avg_shortage_cost = SHORTAGE_COST * AREA_SHORTAGE / NUM_MONTHS

    print(f"\n({SMALL_S:3d},{BIG_S:3d}){avg_ordering_cost + avg_holding_cost + avg_shortage_cost:15.2f}{avg_ordering_cost:15.2f}{avg_holding_cost:15.2f}{avg_shortage_cost:15.2f}")

def update_time_avg_stats():
    # compute time since last event, and update last-event-time marker
    global SIM_TIME, TIME_LAST_EVENT
    time_since_last_event = SIM_TIME - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_TIME

    # determine status of inventory level during previous interval
    global INV_LEVEL, AREA_HOLDING, AREA_SHORTAGE
    if INV_LEVEL < 0:
        AREA_SHORTAGE -= INV_LEVEL * time_since_last_event
    elif INV_LEVEL > 0:
        AREA_HOLDING += INV_LEVEL * time_since_last_event


NUM_EVENTS = 4
def timing():
    """Compare the timings of the next of each possible event type to find
    the next event. Set NEXT_EVENT_TYPE accordingly, and advance the sim
    clock to the time of occurrent of this event.
    """
    global NEXT_EVENT_TYPE, NUM_EVENTS, TIME_NEXT_EVENT, SIM_TIME

    min_time_next_event = np.inf

    for i in range(NUM_EVENTS):
        if TIME_NEXT_EVENT[i] < min_time_next_event:
            min_time_next_event = TIME_NEXT_EVENT[i]
            NEXT_EVENT_TYPE = i

    SIM_TIME = TIME_NEXT_EVENT[NEXT_EVENT_TYPE]


In [55]:
BIG_S = 40
SMALL_S = 20

SETUP_COST = 32.0
INCREMENTAL_COST = 3.0
SHORTAGE_COST = 5.0
HOLDING_COST = 1.0
NUM_MONTHS = 120

# init
initialize()

policies = [(20, 40), (20, 60), (20, 80), (20, 100), (40, 60), (40, 80), (40, 100), (60, 80), (60, 100)]

# Write report header
print("Single-product inventory system\n")
print(f"Initial inventory level{INITIAL_INV_LEVEL:24d}\n")
print(f"Length of the simulation{NUM_MONTHS:23d} months\n")
print(f"K = {SETUP_COST:6.1f}   i ={INCREMENTAL_COST:6.1f}   h ={HOLDING_COST:6.1f}   pi ={SHORTAGE_COST:6.1f}\n")

print(f"          Average           Average           Average            Average\n")
print(f"    Policy      total cost        ordering cost          holiding cost        shortage cost")

for policy in policies:
    SMALL_S, BIG_S = policy
    initialize()
    while True:
        # determine the next event
        timing()

        # update statistical accumulators
        update_time_avg_stats()

        # invoke event function
        match NEXT_EVENT_TYPE:
            case 0:
                order_arrival()
            case 1:
                demand()
            case 3:
                evaluate()
            case 2:
                report()
                break

Single-product inventory system

Initial inventory level                      10

Length of the simulation                    120 months

K =   32.0   i =   3.0   h =   1.0   pi =   5.0

          Average           Average           Average            Average

    Policy      total cost        ordering cost          holiding cost        shortage cost

( 20, 40)         120.67          96.01           9.36          15.30

( 20, 60)         125.73          92.04          16.41          17.28

( 20, 80)         118.16          82.40          27.97           7.79

( 20,100)         125.89          80.50          35.54           9.85

( 40, 60)         125.04          96.41          25.74           2.89

( 40, 80)         127.16          91.16          34.24           1.77

( 40,100)         129.83          83.71          44.83           1.29

( 60, 80)         143.12          96.69          46.41           0.02

( 60,100)         144.31          90.03          54.25           0.04
